# Tarea 3: realizar el procesamiento de datos con SageMaker Processing y el contenedor scikit-learn integrado

En este cuaderno, configure el entorno necesario para ejecutar un script de scikit-learn usando una imagen de Docker proporcionada y mantenida por SageMaker Processing. 

Luego, use la clase **SKLearnProcessor** del SDK para Python de SageMaker para definir y ejecutar un trabajo de procesamiento de scikit-learn.

Por último, valide los resultados del procesamiento de datos guardados en Amazon Simple Storage Service (Amazon S3).

**Nota:** El script de procesamiento realiza un procesamiento de datos básico, como quitar duplicados, transformar la columna objetivo en una que contiene dos etiquetas y hacer una codificación one-hot y una división 80-20 para producir conjuntos de datos de entrenamiento y pruebas. 

## Tarea 3.1: configuración del entorno

En esta tarea, instale los paquetes y las dependencias necesarios. 

Configure un bucket de Amazon S3 para almacenar las salidas del trabajo de procesamiento y obtenga el rol de ejecución para ejecutar el trabajo de SageMaker Processing.

In [ ]:
#install-dependencies
import logging
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sagemaker.s3 import S3Downloader

sagemaker_logger = logging.getLogger("sagemaker")
sagemaker_logger.setLevel(logging.INFO)
sagemaker_logger.addHandler(logging.StreamHandler())

sagemaker_session = sagemaker.Session()

#Execution role to run the SageMaker Processing job
role = sagemaker.get_execution_role()
print("SageMaker Execution Role: ", role)

#S3 bucket to read the SKLearn processing script and writing processing job outputs
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)

## Tarea 3.2: ejecutar el trabajo de SageMaker Processing

En esta tarea, importe y revise el conjunto de datos preprocesado.

In [ ]:
#import-data
prefix = 'data/input'

S3Downloader.download(s3_uri=f"s3://{bucket}/{prefix}/adult_data.csv", local_path= 'data/')

shape=pd.read_csv("data/adult_data.csv", header=None)
shape.sample(5)

Para este laboratorio, se realizarán las transformaciones de datos, como eliminar los duplicados, transformar la columna objetivo en una columna que contiene dos etiquetas y hacer la codificación one-hot de las funciones categóricas.

Luego, cree una clase SKLearnProcessor para definir y ejecutar un script de procesamiento de scikit-learn como un trabajo de procesamiento. Consulte [SKLearnProcessor para scikit-learn Sagemaker](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#sagemaker.sklearn.processing.SKLearnProcessor) para obtener más información sobre esta clase.

Para crear la clase SKLearnProcessor, configure los siguientes parámetros:
- **base_job_name**: prefijo para el nombre del trabajo de procesamiento
- **framework_version**: versión de scikit-learn
- **role**: rol de ejecución de SageMaker
- **instance_count**: cantidad de instancias para ejecutar el trabajo de procesamiento
- **instance_type**: tipo de instancia de Amazon Elastic Compute Cloud (Amazon EC2) usada para el trabajo de procesamiento

In [ ]:
#scikit-learn-processor
import os
from sagemaker.sklearn.processing import SKLearnProcessor

# create a SKLearnProcessor
sklearn_processor = SKLearnProcessor(
    base_job_name="sklearn-preprocessor",
    framework_version="0.20.0", 
    role=role, 
    instance_count=1,
    instance_type="ml.m5.xlarge"
)


A continuación, use el método **SKLearnProcessor.run()** para ejecutar un script **sklearn_preprocessing.py** como trabajo de procesamiento. 

Para ejecutar el trabajo de procesamiento, configure los siguientes parámetros:
- **code**: ruta del script de preprocesamiento 
- **inputs and outputs**: ruta de entradas y salidas para el script de preprocesamiento (ubicaciones de entradas y salidas de Amazon S3)
- **arguments**: argumentos de línea de comandos para el script de preprocesamiento (como una relación de división para entrenamiento y prueba)

El tiempo para completar este trabajo de procesamiento es de aproximadamente 4 a 5 minutos. Mientras se ejecuta el trabajo, puede revisar la fuente para el script del preprocesamiento (que se configuró como parte de este laboratorio) abriendo el archivo **sklearn_preprocessing.py** desde el navegador de archivos.

In [ ]:
#processing-job
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Amazon S3 path prefix
input_raw_data_prefix = "data/input"
output_preprocessed_data_prefix = "data/output"
scripts_prefix = "scripts/smstudiofiles"
logs_prefix = "logs"

# Run the processing job
sklearn_processor.run(
    code="s3://" + os.path.join(bucket, scripts_prefix, "sklearn_preprocessing.py"),
    inputs=[ProcessingInput(source="s3://" + os.path.join(bucket, input_raw_data_prefix, "adult_data.csv"),
                            destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", 
                         source="/opt/ml/processing/train",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "train")),
        ProcessingOutput(output_name="test_data", 
                         source="/opt/ml/processing/test",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "test")),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)

print("SKLearn Processing Job Completed.")

## Tarea 3.3: validar los resultados del procesamiento de datos

En esta tarea, debe validar la salida del trabajo de procesamiento que ejecutó mediante la revisión de las primeras cinco filas de los conjuntos de datos de salida de entrenamiento y prueba.

In [ ]:
#view-train-dataset
print("Top 5 rows from s3://{}/{}/train/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/train/train_features.csv - | head -n5

In [ ]:
#view-validation-dataset
print("Top 5 rows from s3://{}/{}/validation/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/test/test_features.csv - | head -n5

### Conclusión

¡Felicitaciones! Usó SageMaker Processing para crear correctamente un trabajo de procesamiento de scikit-learn usando el SDK para Python de SageMaker y ejecutó el trabajo de procesamiento.

La siguiente tarea del laboratorio se enfoca en el procesamiento de datos usando SageMaker Processing con su contenedor de procesamiento.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Tarea 4: realizar el procesamiento de datos con su propio contenedor**.